[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

Install and Import required libraries

In [28]:
!pip install PyGithub
!!pip install dnspython

import pymongo
from github import Github
import pandas as pd
import json
import numpy as np
import datetime as ddt
from datetime import datetime

#making pandas to allow us to print to our satisfaction :P
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.options.mode.chained_assignment = None

Connect to Github and fetch the source raw data file names                      
*   Data source : CSSEGISandData/COVID-19



In [29]:
#Connect to github's api service 
g= Github()

# Getting the csv file names and a link to their raw source, contatining the COVID-19 raw data
repo = g.get_repo("CSSEGISandData/COVID-19")
cont = repo.get_contents("/csse_covid_19_data/csse_covid_19_daily_reports")
all_files = []
for i in range(1,len(cont)):
  url = cont[i].download_url
  #avoiding data before 31st March 2020 since they were not formatted as expected
  if (".csv" in url) and (datetime(int(url[118:122]),int(url[112:114]),int(url[115:117])) > datetime(2020,3,31)) and (datetime(int(url[118:122]),int(url[112:114]),int(url[115:117])) < datetime(2021,1,15)):
    all_files.append(url)

Download all the source files and feed them to a Pandas Dataframe

In [30]:
## Reading the csv files into a pandas data frame
li = []
for filename in all_files:
  if ".csv" in filename:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

CovidData = pd.concat(li, axis=0, ignore_index=True)

CovidData

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-02 05:22:33,33.939110,67.709953,51526,2191.0,41727.0,0.0,Afghanistan,0.000000,4.25222,NaN,NaN
1,NaN,NaN,NaN,Albania,2021-01-02 05:22:33,41.153300,20.168300,58316,1181.0,33634.0,23501.0,Albania,2026.409062,2.02517,NaN,NaN
2,NaN,NaN,NaN,Algeria,2021-01-02 05:22:33,28.033900,1.659600,99897,2762.0,67395.0,29740.0,Algeria,227.809861,2.76485,NaN,NaN
3,NaN,NaN,NaN,Andorra,2021-01-02 05:22:33,42.506300,1.521800,8117,84.0,7463.0,570.0,Andorra,10505.403482,1.03487,NaN,NaN
4,NaN,NaN,NaN,Angola,2021-01-02 05:22:33,-11.202700,17.873900,17568,405.0,11146.0,6017.0,Angola,53.452981,2.30533,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085672,NaN,NaN,NaN,Vietnam,2021-01-01 05:23:07,14.058324,108.277199,1465,35.0,1325.0,105.0,Vietnam,1.505056,2.38908,NaN,NaN
1085673,NaN,NaN,NaN,West Bank and Gaza,2021-01-01 05:23:07,31.952200,35.233200,138004,1400.0,117183.0,19421.0,West Bank and Gaza,2705.209691,1.01446,NaN,NaN
1085674,NaN,NaN,NaN,Yemen,2021-01-01 05:23:07,15.552727,48.516388,2099,610.0,1394.0,95.0,Yemen,7.037492,29.0615,NaN,NaN
1085675,NaN,NaN,NaN,Zambia,2021-01-01 05:23:07,-13.133897,27.849332,20725,388.0,18660.0,1677.0,Zambia,112.734169,1.87214,NaN,NaN


Clean the data frame

In [31]:
#Data Pre-Processing 
CovidData['Last_Update']= pd.to_datetime(CovidData['Last_Update'])
CovidData['Incident_Rate'] = CovidData['Incident_Rate'].fillna(CovidData['Incidence_Rate'])
CovidData['Incident_Rate'] = CovidData['Incident_Rate'].fillna(0)
CovidData['Case_Fatality_Ratio'] = CovidData['Case_Fatality_Ratio'].fillna(CovidData['Case-Fatality_Ratio'])

# Extract the Latitude and Longitude
Lat_Long = pd.DataFrame()
Lat_Long["Country"] = CovidData["Country_Region"]
Lat_Long["Lat"] = CovidData["Lat"]
Lat_Long["Long_"] = CovidData["Long_"]
Lat_Long = Lat_Long.groupby(["Country"])
Lat_Long = Lat_Long.first()

CovidData = CovidData.drop(['FIPS','Admin2','Lat','Long_','Incidence_Rate', 'Case-Fatality_Ratio'], axis=1)

# Countries
CountryList = CovidData.Country_Region.unique()

CovidData

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,Afghanistan,2021-01-02 05:22:33,51526,2191.0,41727.0,0.0,Afghanistan,0.000000,4.25222
1,NaN,Albania,2021-01-02 05:22:33,58316,1181.0,33634.0,23501.0,Albania,2026.409062,2.02517
2,NaN,Algeria,2021-01-02 05:22:33,99897,2762.0,67395.0,29740.0,Algeria,227.809861,2.76485
3,NaN,Andorra,2021-01-02 05:22:33,8117,84.0,7463.0,570.0,Andorra,10505.403482,1.03487
4,NaN,Angola,2021-01-02 05:22:33,17568,405.0,11146.0,6017.0,Angola,53.452981,2.30533
...,...,...,...,...,...,...,...,...,...,...
1085672,NaN,Vietnam,2021-01-01 05:23:07,1465,35.0,1325.0,105.0,Vietnam,1.505056,2.38908
1085673,NaN,West Bank and Gaza,2021-01-01 05:23:07,138004,1400.0,117183.0,19421.0,West Bank and Gaza,2705.209691,1.01446
1085674,NaN,Yemen,2021-01-01 05:23:07,2099,610.0,1394.0,95.0,Yemen,7.037492,29.0615
1085675,NaN,Zambia,2021-01-01 05:23:07,20725,388.0,18660.0,1677.0,Zambia,112.734169,1.87214


Cook JSONs from the Pandas array as required [Long running section]

In [32]:
#Cooking the json objects
country_wise = []
for country_value in CountryList:
  country_filter = CovidData[CovidData['Country_Region']==country_value]
  country_filter['Last_Update'] = country_filter['Last_Update'].dt.date
  country_filter = country_filter.groupby(['Country_Region','Last_Update'],as_index=False).agg({'Confirmed': 'sum', 'Deaths': 'sum',  'Recovered': 'sum','Active': 'sum','Incident_Rate': 'first'})

  num = country_filter.loc[:,['Confirmed','Deaths','Recovered']]
  difference = num.diff(axis=0)
  difference['Last'] = country_filter['Last_Update']
  country_filter["New_Cases"]=difference["Confirmed"]
  country_filter["New_Deaths"]=difference["Deaths"]
  country_filter["New_Recoveries"]=difference["Recovered"]

  country_filter["Case_Fatality_Rate"]=country_filter["Deaths"]/country_filter["Confirmed"]
  country_filter["Case_Fatality_Rate"]=country_filter["Case_Fatality_Rate"].replace([np.inf, -np.inf], 0)
  country_filter["Case_Fatality_Rate"]=country_filter["Case_Fatality_Rate"].replace(np.nan, 0)

  country_filter['Last_Update'] = pd.to_datetime(country_filter['Last_Update'])
  # country_filter['Last_Update'] = country_filter['Last_Update'].dt.normalize() + ddt.timedelta(hours=2)
  country_filter['Last_Update'] = pd.to_datetime(country_filter['Last_Update']).dt.tz_localize('US/Eastern').dt.tz_convert('US/Eastern')


  country_filter['Incident_Rate'] = country_filter['Incident_Rate'].round(4)
  country_filter['Case_Fatality_Rate'] = country_filter['Case_Fatality_Rate'].round(4)
  country_filter.loc[0,['New_Cases','New_Deaths','New_Recoveries']]=0

  if ( np.sum(country_filter["New_Cases"].isna() ) > 0 or np.sum(country_filter["New_Deaths"].isna())>0 or np.sum(country_filter["New_Recoveries"].isna())>0 or np.sum(country_filter["Case_Fatality_Rate"].isna())>0):
    print("Culprited!")
    print(country_value)

  temp_json = country_filter.to_dict(orient='records')
  pack_jsons = {
      "country":country_value,
      "latitude" : Lat_Long.loc[country_value].Lat,
      "longitude" : Lat_Long.loc[country_value].Long_,
      "object":temp_json
  }
  country_wise.append(pack_jsons)



Connect to Mongodb and dump the JSONs

In [ ]:
# Connecting to the mongo db
client = pymongo.MongoClient("MONGODB_URL") #replace MONGODB_URL with your mongo cluster connection link
db = client["covid"]
col = db["countrywise"]
col.drop()
db = client["covid"]
col = db["countrywise"]

#inserting the bulk object to db
insert_return = col.insert_many(country_wise)